In [ ]:
import os
from qiskit import *
from qiskit import QuantumCircuit, Aer
import qiskit.circuit.library as library
import sympy
import numpy as np
from numpy import pi
from scipy import linalg
from functools import reduce
import itertools

from qc_utils.gates import *
import qc_utils.idx as idx
import decomp

import sympy

In [ ]:
def run_small():
    fids = []
    for dir in os.listdir('QASMBench/small'):
        try:
            print('='*80)
            u = f'QASMBench/small/{dir}/{dir}.qasm'
            print(dir)
            d = decomp.Decomposer(u, False)
            u = d.unitary

            d.reconstruct_with_CX()
            qc = d.circuit
            backend = Aer.get_backend('unitary_simulator')
            job = execute(qc, backend)
            result = job.result()
            unitary = np.array(result.get_unitary(qc, decimals=10))[:u.shape[0], :u.shape[0]]
            print(fid(u,unitary))
            fids.append((dir, fid(u,unitary)))
            #print('SUCCESS,' if np.all(np.isclose(u, unitary)) else 'FAIL,', 'fidelity:', fid(u, unitary))
            #display(sympy.Matrix(unitary))
            #display(qc.draw(fold=150))
        except Exception as e:
            print(e)
    return fids
#f = run_small()

In [ ]:
import os
from functools import reduce
#u = np.kron(Z, np.kron(X,Y))
u = "QASMBench/small/iswap_n2/iswap_n2.qasm"
#u = random_u(2)
d = decomp.Decomposer(u, optimize=False, csdstop=2)
u0 = d.unitary
nbits = int(np.log2(u0.shape[0]))
#display(sympy.Matrix(u0))

print('='*150)

d.reconstruct_with_CX()
print('Done')

size = 2**nbits

backend = Aer.get_backend('unitary_simulator')
job = execute(d.circuit, backend)
result = job.result()
unitary = np.array(result.get_unitary(d.circuit, decimals=10))[:size, :size]
print(fid(u0,unitary))
#display(sympy.Matrix(unitary))
if nbits <= 3:
    display(d.circuit.draw(fold=150))

d = decomp.Decomposer(u, optimize=False)
u0 = d.unitary
nbits = int(np.log2(u0.shape[0]))
#display(sympy.Matrix(u))

print('='*150)

d.reconstruct_with_CU()
print('Done')

size = 2**nbits

backend = Aer.get_backend('unitary_simulator')
job = execute(d.circuit, backend)
result = job.result()
unitary = np.array(result.get_unitary(d.circuit, decimals=10))[:size, :size]
print(fid(u0,unitary))
#display(sympy.Matrix(unitary))
if nbits <= 3:
    display(d.circuit.draw(fold=150))

We can decompose the multi-controlled U gate into three $C^{n-1}X$ gates (and then to $C^{n-1}Z$ gates) through the following:

In [ ]:
from qiskit.circuit import Gate, ControlledGate
import qiskit.circuit
from qiskit.circuit.library import ZGate, PhaseGate
from qiskit.quantum_info import Operator

ccz = ZGate().control(2)

u = Operator([[1,0],[0,1]]).to_instruction().control(2)
u.name = 'U'
qc = QuantumCircuit(3)
qc.append(u, [0,1,2])
display(qc.draw())

qc = QuantumCircuit(3)
qc.append(Gate(name='C', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='B', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='A', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='U^t(0,-a,0)', num_qubits=1, params=[]), [2])
qc.toffoli(0,1,2)
qc.append(Gate(name='U(0,-a,0)', num_qubits=1, params=[]), [2])
qc.append(PhaseGate(pi/2).control(2), [0,1,2])
qc.draw(fold=-1)

where $U = e^{ia}AXBXC$ (N&C page 180). The last five gates correspond to a controlled $e^{i \alpha}$ phase applied to the target bit.

From here, this can easily be converted to CZ gates with $A' = HA, B' = HBH, C' = CH, U' = UH$

In [ ]:
from qiskit.compiler import transpile
qc = QuantumCircuit.from_qasm_file('QASMBench/small/dnn_n2/dnn_n2.qasm')
transpile(qc, basis_gates=['u3', 'cx'], optimization_level=3).draw()

In [ ]:
pauli_decomp(np.kron(X,Z))

In [ ]:
pauli_reconstruct(pauli_decomp(np.kron(X,Z)), 2)

In [ ]:
np.kron(X,Z)